In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

import numpy as np
import os


In [10]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,Dense,Embedding
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences

tf.__version__

'2.3.0'

### Embedding layer in Tensorflow

The Embedding layer takes at least two arguments:
- The number of samples (i.e. words, tokens)
- The dimensionality (length) of the embedding vector

This layer returns a 3D floating point tensor, of shape (num_samples, sequence_length, embedding_dimensionality). 

When this layer is instantiated, its weights are initially random. During training, these word vectors will be gradually adjusted via backpropagation, structuring the space into a representation that can be used in other layers.



### Sentiment prediction using IMDB Movie Reviews

https://www.tensorflow.org/datasets/catalog/imdb_reviews

The number of words is restricted to the top 10,000 most common words in the reviews.

The reviews are truncated to only 20 words. 

The model will consist of:

- An 8-dimensional embedding for each of the 10,000 words
- Flatten layer the tensor to 2D, and  
- A single Dense layer to classify a review as favorable or not.

In [11]:
max_features = 10000 # Number of words to consider as features

maxlen = 20 # Cut text after maxlen (among top max_features most common words)

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
type(X_train[0]),len(X_train[0]),max(X_train[0])

X_train[0][0:10]

17465344/17464789 [==============================] - 0s 0us/step
(25000,) (25000,) (25000,) (25000,)


[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65]

In [12]:
len(X_train[1])

189

#### Create 2D integer tensor of shape (samples, maxlen)

In [13]:
X_train = pad_sequences(X_train, maxlen=maxlen)
X_test = pad_sequences(X_test, maxlen=maxlen)
print(X_train.shape,X_test.shape)
X_train[0,:]

(25000, 20) (25000, 20)


array([  65,   16,   38, 1334,   88,   12,   16,  283,    5,   16, 4472,
        113,  103,   32,   15,   16, 5345,   19,  178,   32], dtype=int32)

#### Create Model


In [14]:
model = Sequential()

model.add(Embedding(10000, 8, input_length=maxlen))
# Output shape (samples, maxlen, 8)
model.add(Flatten()) # Flatten into a 2D tensor of shape (samples, maxlen * 8)
model.add(Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 8)             80000     
_________________________________________________________________
flatten (Flatten)            (None, 160)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 161       
Total params: 80,161
Trainable params: 80,161
Non-trainable params: 0
_________________________________________________________________


#### Compile and fit

In [15]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

history = model.fit(X_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)

Epoch 1/10
625/625 [==============================] - 1s 2ms/step - loss: 0.6762 - acc: 0.6004 - val_loss: 0.6314 - val_acc: 0.6892
Epoch 2/10
625/625 [==============================] - 1s 2ms/step - loss: 0.5505 - acc: 0.7459 - val_loss: 0.5306 - val_acc: 0.7276
Epoch 3/10
625/625 [==============================] - 1s 2ms/step - loss: 0.4657 - acc: 0.7850 - val_loss: 0.4999 - val_acc: 0.7464
Epoch 4/10
625/625 [==============================] - 1s 2ms/step - loss: 0.4272 - acc: 0.8043 - val_loss: 0.4933 - val_acc: 0.7504
Epoch 5/10
625/625 [==============================] - 1s 2ms/step - loss: 0.4029 - acc: 0.8184 - val_loss: 0.4948 - val_acc: 0.7528
Epoch 6/10
625/625 [==============================] - 1s 2ms/step - loss: 0.3838 - acc: 0.8288 - val_loss: 0.4980 - val_acc: 0.7554
Epoch 7/10
625/625 [==============================] - 1s 2ms/step - loss: 0.3667 - acc: 0.8382 - val_loss: 0.4996 - val_acc: 0.7590
Epoch 8/10
625/625 [==============================] - 1s 2ms/step - loss: 0.

#### Compute classification accuracy

In [16]:
def accuracy(x,y):
    y_hat = model.predict(x)
    foo = np.array([1 if i > .5 else 0 for i in y_hat[:,0] ])
    return np.sum(foo == y)/len(y)


In [17]:
accuracy(X_train,y_train),accuracy(X_test,y_test)

(0.85516, 0.76136)

This model with a single Dense layer treats each word in the input sequence separately. Inter-word relationships and structure sentence are not taken into account.

We could add recurrent layers or 1D convolutional layers after the embedded sequences to learn features that take into account each sequence as a whole.